In [3]:
from neo4j import GraphDatabase
from tqdm import tqdm

In [6]:
uri = ""
username = ""
password = ""

In [7]:
import pandas as pd
df = pd.read_csv("./data/entities/movies.csv")
df

,name,id,url,rate,length
0,我是路人甲,0,http://movie.douban.com/subject/25746375/,7.4,134.0
1,铁拳,1,http://movie.douban.com/subject/5446197/,7.1,123.0
2,我们梦中见,2,http://movie.douban.com/subject/25885212/,7.6,92.0
3,少年透明人,3,http://movie.douban.com/subject/25728581/,6.6,100.0
4,撒迦利亚,4,http://movie.douban.com/subject/5156558/,6.0,95.0
...,...,...,...,...,...
4582,幽灵世界,4582,http://movie.douban.com/subject/1304868/,7.9,111.0
4583,吮拇指的人,4583,http://movie.douban.com/subject/1422954/,7.2,96.0
4584,歌舞青春3：毕业季,4584,http://movie.douban.com/subject/2215609/,7.3,112.0
4585,大河恋,4585,http://movie.douban.com/subject/1292718/,8.4,123.0


In [8]:
driver = GraphDatabase.driver(uri, auth=(username, password))

In [9]:
def execute_query(tx, query, parameters=None):
    tx.run(query, parameters)

In [10]:
def special_word(strs):
    rgx = ["'",'=~','~', '*', '%', '^', '<=', '>=', '=', '<>', '!=', 'XOR', 'OR', '|', '\\']
    for rg in rgx:
        strs = strs.replace(rg, ' ')
    strs.replace('\'', '')
    return strs

In [11]:
['1','2','3']

['1', '2', '3']

In [13]:
','.join(['1','2','3'])

'1,2,3'

In [ ]:
with driver.session() as session:
    for row in tqdm(range(len(df))):
        sx_dic = {}
        for sx in ['id','url','rate','length']:
            if df.loc[row][sx]:
                if sx == 'id':
                    sx_dic['code'] = df.loc[row][sx]
                else:
                    sx_dic[sx] = df.loc[row][sx]
        query = "CREATE (p:"
        query += "Movie {"
        query += "name:'" + special_word(str(df.loc[row]['name'])) + "',"
        query+= ','.join([f"{k} :  '{v}'" for k, v in sx_dic.items()])
        query += "})"
        try:
            # 执行查询
            session.write_transaction(execute_query, query)
        except Exception as e:
            print(e)
        
        # CREATE(<node_name>:<label_name> {<property1_name>:<property1_value>,...,<propertyn_name>:<propertyn_value>) 

In [ ]:
def create_node(name_str):
    df = pd.read_csv('./data/entities/'+name_str+'.csv')
    with driver.session() as session:
        for row in tqdm(range(len(df))):
            query = "CREATE (p:" + name_str.rstrip('s').title()
            query += " {name:'" + special_word(str(df.loc[row]['name'])) + "',"
            query += "code:'" + str(df.loc[row]['id'])
            query += "'})"
            try:
                # 执行查询
                session.write_transaction(execute_query, query)
            except Exception as e:
                print(e)

In [ ]:
create_node('years')

In [ ]:
create_node('districts')

In [ ]:
create_node('mv_classes')

In [ ]:
create_node('languages')

In [ ]:
create_node('people')

In [ ]:
import pandas as pd
relation_df = pd.read_csv('./data/relations/triple.csv')
relation_df

In [ ]:
for row in tqdm(range(len(relation_df))):
    start_node = str(relation_df.loc[row]['Entity1'])
    end_node = str(relation_df.loc[row]['Entity2'])           
    # 查找已存在的节点
    with driver.session() as session:
        s1 = " (a:Movie {code: $code1})"
        if relation_df.loc[row]['Relation'] in ['导演','编剧','演员']:
            s2 = " (b:People {code: $code2})"
        elif relation_df.loc[row]['Relation'] == '上映时间':
            s2 = " (b:Year {code: $code2})"
        elif relation_df.loc[row]['Relation'] == '上映地区':
            s2 = " (b:District {code: $code2})"
        elif relation_df.loc[row]['Relation'] == '类型':
            s2 = " (b:Mv_Classe {code: $code2})"
        elif relation_df.loc[row]['Relation'] == '语言':
            s2 = " (b:Language {code: $code2})"

    # 创建关系
    with driver.session() as session:
        try:
            session.run("MATCH"+s1+","+s2+""
                        "CREATE (a) - [:"+str(relation_df.loc[row]['Relation'])+"] -> (b)",
                        code1=start_node,code2=end_node)
        except Exception as e:
            print("create relation error:" , e)